In [14]:
sc.addPyFile("s3://mist-data-science-dev/ruchitm/spark_testing/spark_jobs_ruchit.zip")
from analytics.utils.bytes_util import convert_bytes_to_mac, convert_bytes_to_uuid

from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta
from functools import partial
import math
from pprint import pprint

def round_datetime(timestamp, min_interval=0, sec_interval=0):
    if timestamp : 
        timestamp = int(timestamp)
    else :
        timestamp=0
    ## scaling timestamp to seconds resolution
    scale=1
    resolution = int(math.log10(timestamp))
    if resolution==12:
        scale=1E3
    elif resolution==15:
        scale=1E6
    elif resolution==18:
        scale=1E9
    timestamp = timestamp//scale
    tm = datetime.fromtimestamp(timestamp)
    if min_interval > 0 and sec_interval > 0:
        tm = tm - timedelta(minutes=tm.minute % min_interval, seconds=tm.second % sec_interval)
    elif min_interval > 0 :
        tm = tm - timedelta(minutes=tm.minute % min_interval, seconds=tm.second)
    elif sec_interval > 0 :
        tm = tm - timedelta(minutes=tm.minute, seconds=tm.second % sec_interval)
    return tm

def date_list(endDate, delta=14):
    temp=[endDate]
    for i in range(1,delta+1):
        temp.append(endDate - timedelta(days=i))
    return '{' + ','.join([str(d.date()) for d in temp]) + '}'

@F.udf(IntegerType())
def ts_resolution(timestamp):
    return int(math.log10(timestamp))+1


udf_rounddate = F.udf(round_datetime, TimestampType())
udf_round_1hour_floor = F.udf(partial(round_datetime, min_interval=60), TimestampType())
udf_round_10min_floor = F.udf(partial(round_datetime, min_interval=10), TimestampType())
udf_round_1min_floor = F.udf(partial(round_datetime, min_interval=1), TimestampType())
udf_round_3min_floor = F.udf(partial(round_datetime, min_interval=3), TimestampType())
udf_round_30sec_floor = F.udf(partial(round_datetime, min_interval=1, sec_interval=30), TimestampType())

time_agg_map = {"When_1min": udf_round_1min_floor,
               "When_30sec": udf_round_30sec_floor,
               "When_10min": udf_round_10min_floor,
               "When_3min": udf_round_3min_floor,
               "When_60min": udf_round_1hour_floor}

def magic_median(varname):
    return F.expr(f'percentile_approx({varname}, 0.5)')

def magic_iqr(varname):
    return F.expr(f'percentile_approx({varname}, 0.75)-percentile_approx({varname}, 0.25)')

def getDateString(dates):
    if ',' not in dates:
        return dates.split('{')[1].split('}')[0]
    else:
        return dates.split('{')[1].split(',')[0] + "_" + dates.split('{')[1].split(',')[-1].split('}')[0]

def get_agg_funcs(data_cols):
    avg_funcs = [F.avg(col).alias(f'avg_{col}') for col in data_cols]
    std_funcs = [F.stddev(col).alias(f'std_{col}') for col in data_cols]
    median_funcs = [magic_median(col).alias(f'median_{col}') for col in data_cols]
    iqr_funcs = [magic_iqr(col).alias(f'iqr_{col}') for col in data_cols]
    return avg_funcs + std_funcs + median_funcs + iqr_funcs

@F.udf(returnType=FloatType())
def calculate_loss(mark_count, ack_count):
    if mark_count>0.:
        return 100 * (mark_count-ack_count)/mark_count
    return -99.

@F.udf(returnType=FloatType())
def calculate_tagged_pct(conf_count, mark_count):
    if conf_count>0.:
        return 100 * mark_count/conf_count
    return -99.

@F.udf(returnType=StringType())
def udf_convert_bytes_to_uuid(uuid):
    return convert_bytes_to_uuid(uuid)


@F.udf(returnType=StringType())
def udf_convert_bytes_to_mac(mac_id):
    return convert_bytes_to_mac(mac_id)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
############################################################
endDate = datetime.today()
endDate = datetime(2022, 8, 3)
interval = 0
dates=date_list(endDate, delta=interval)
date_string = getDateString(dates)
hour = '{*}'
print(date_string, '', hour)
storage = 's3'
# env = 'production'
env = 'staging'
source = 'oc-stats-analytics'
username = 'ruchitm' # change to personal folder name
outpath = f'{storage}://mist-data-science-dev/{username}'

cols = ["when","site_id","mac","firmware_version","t128.router_name",
        "t128.node_name","t128.plugin_version","interfaces","org_id","model","ssr_peer_path_stats"]

##### Define aggregation window #####
#time_agg_col = 'When_30sec'
time_agg_col = 'when'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2022-08-03  {*}

## Process peer-path data

In [18]:
FPM_PLUGIN_VERSION = '' #'3.2.0-207'
FPM_FW_VERSION = '' #'5.5.0-0'

SPOKE_MACS = ['020001e9707f'] #['020001e9707f', '0200010777d4', '020001f339eb']
interface = 'WAN'

bfd_group_cols = [time_agg_col,'site_id','peer_path_named','mac','device_interface', 'router_name']
fpm_group_cols = [time_agg_col,'site_id','peer_path_named','mac','device_interface','router_name',
                  'fpm_traffic_class','fpm_protocol']

bfd_cols = [f'bfd_{metric}' for metric in ['latency','jitter','loss']]                  
fpm_cols = [f'fpm_{metric}' for metric in ['latency','jitter','tx_loss', 'rx_loss', 'tx_mark_delta', 'rx_mark_delta']]

df = spark.read.parquet(f"{storage}://mist-secorapp-{env}/{source}/{source}-{env}/dt={dates}/hr={hour}")
df = df.select(cols).filter(F.col('model').startswith('SSR'))\
        .withColumn('when', udf_rounddate(F.col('when')))


df_peer_path = df.filter(F.size('ssr_peer_path_stats')>0)

if FPM_PLUGIN_VERSION:
    df_peer_path = df_peer_path.filter((F.col('plugin_version')>=FPM_PLUGIN_VERSION))

if FPM_FW_VERSION:
    df_peer_path = df_peer_path.filter((F.col('firmware_version')>=FPM_FW_VERSION))

if len(SPOKE_MACS)>0:
    df_peer_path = df_peer_path.filter(F.col('mac').isin(SPOKE_MACS))

### Explode interfaces ### 
INFO_COLS = ['when', 'site_id','mac','router_name', 'node_name','org_id','firmware_version','plugin_version']

df_intf = df_peer_path.select(*INFO_COLS, F.explode('interfaces').alias('interfaces'))
df_intf = df_intf.select(*INFO_COLS, F.col('interfaces.name').alias('network_interface'),
                                   'interfaces.rx_bytes', 'interfaces.rx_packets',
                                    'interfaces.tx_bytes', 'interfaces.tx_packets')
df_intf = df_intf.filter(F.col('network_interface')==interface)

### Explode ssr_peer_path_stats ###
df_peer_path = df_peer_path.select(*INFO_COLS, F.explode('ssr_peer_path_stats').alias('ssr_peer_path_stats'))
df_peer_path = df_peer_path.select(*INFO_COLS, 'ssr_peer_path_stats.*')
df_peer_path = df_peer_path.filter(F.col('network_interface')==interface)

### Explode samples ###
other_cols = [col for col in df_peer_path.columns if col!='samples']
df_peer_path = df_peer_path.select(*other_cols,
                                   F.size('samples').alias('n_samples'),
                                   F.explode(F.col('samples')).alias('samples'))

df_peer_path = df_peer_path.select(*other_cols, F.col('samples.*'),'n_samples') \
                           .withColumnRenamed('latency','bfd_latency') \
                           .withColumnRenamed('jitter','bfd_jitter')\
                           .withColumnRenamed('loss','bfd_loss')

# df_peer_path = df_peer_path.withColumn(time_agg_col, time_agg_map[time_agg_col](F.col('timestamp')))

# df_peer_path = df_peer_path.filter(F.col('is_active')==True)
# df_peer_path = df_peer_path.filter(F.col('is_up')==True)

df_peer_path = df_peer_path.withColumn('peer_path', F.concat_ws("__", 'mac', 'network_interface',
                                                                'peer_mac','adjacent_address'))
df_peer_path = df_peer_path.withColumn('peer_path_named', F.concat_ws("__", 'router_name', 'network_interface',
                                                                      'peer_name','adjacent_address'))


df_bfd_agg = df_peer_path.groupby(bfd_group_cols).agg(*get_agg_funcs(bfd_cols), F.count('bfd_latency').alias('cnt_samples'))

other_cols = [col for col in df_peer_path.columns if col!='fpmstats']

df_peer_path_fpm = df_peer_path.filter(F.size('fpmstats')>0)\
                            .select(*other_cols, F.explode('fpmstats').alias('fpmstats'))

df_peer_path_fpm = df_peer_path_fpm.select(*other_cols, 'fpmstats.*') \
                .withColumnRenamed('latency','fpm_latency') \
                .withColumnRenamed('jitter','fpm_jitter')\
                .withColumnRenamed('traffic_class','fpm_traffic_class')\
                .withColumnRenamed('protocol','fpm_protocol')

df_peer_path_fpm = df_peer_path_fpm.withColumn('fpm_tx_mark_delta', calculate_tagged_pct('fpmconfigured_tx_mark_count', 'tx_mark_count'))\
                        .withColumn('fpm_rx_mark_delta', calculate_tagged_pct('fpmconfigured_rx_mark_count', 'rx_mark_count'))\
                        .withColumn('fpm_tx_loss', calculate_loss('tx_mark_count', 'tx_acknowledged_count'))\
                        .withColumn('fpm_rx_loss', calculate_loss('rx_mark_count', 'rx_acknowledged_count'))

df_fpm_agg = df_peer_path_fpm.groupby(fpm_group_cols).agg(*get_agg_funcs(fpm_cols))

intf_outpath = f"{outpath}/ssr_peer_path/data/{env}/interface_bps_{time_agg_col}/dt={date_string}/"
bfd_outpath = f"{outpath}/ssr_peer_path/data/{env}/bfd_stats_agg_{time_agg_col}/dt={date_string}/"
# fpm_outpath = f"{outpath}/ssr_peer_path/data/{env}/fpm_stats_agg_{time_agg_col.split('_')[1]}/dt={date_string}/"

df_intf.write.csv(intf_outpath, mode="overwrite", header=True)
df_bfd_agg.write.csv(bfd_outpath, mode="overwrite", header=True)
# df_fpm_agg.write.csv(fpm_outpath, mode="overwrite", header=True)

print(intf_outpath)
print(bfd_outpath)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://mist-data-science-dev/ruchitm/ssr_peer_path/data/staging/interface_bps_when/dt=2022-08-03/
s3://mist-data-science-dev/ruchitm/ssr_peer_path/data/staging/bfd_stats_agg_when/dt=2022-08-03/

In [10]:
df_bfd_agg.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+--------------------+------------+----------------+-----------+------------------+--------------+------------------+-------------------+--------------+------------------+------------------+-----------------+---------------+---------------+--------------+------------+---------------+--------------+------------+-----------+
|               when|             site_id|     peer_path_named|         mac|device_interface|router_name|   avg_bfd_latency|avg_bfd_jitter|      avg_bfd_loss|    std_bfd_latency|std_bfd_jitter|      std_bfd_loss|median_bfd_latency|median_bfd_jitter|median_bfd_loss|iqr_bfd_latency|iqr_bfd_jitter|iqr_bfd_loss|cnt_bfd_latency|cnt_bfd_jitter|cnt_bfd_loss|cnt_samples|
+-------------------+--------------------+--------------------+------------+----------------+-----------+------------------+--------------+------------------+-------------------+--------------+------------------+------------------+-----------------+---------------+---